In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Nov  2 14:02:22 2018

@author: Vasudev the Great
"""

In [13]:
import tweepy
import re
from datetime import datetime, timedelta


#Variables that contains the user credentials to access Twitter API 
access_token = "1045380725632106496-m9ARszPzmQ7RXbmiYVfws8azBOVuWJ"
access_token_secret = "VozPM5idWCbNQKFLJpch3GgdbZRDRlhihh1dzL72IsNxp"
consumer_key = "i66bjNNV4Lem9RkEv5R97o5uH"
consumer_secret = "MfnLzhCw7R5niUmk2hwzt3NB8av6Ac4SFk8b8Of8mGdZ6wCLxl"

# creating an OAuthHandler 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

API = tweepy.API(auth)

# creates a dict object for each tweet
results = [status._json for status in tweepy.Cursor(API.search, q="$EURUSD",
                    count=1200, tweet_mode='extended', lang='en').items()]
# creates an empty list
# gets the last 3 days as test data and 7 days before that as training data
train_tweets = []
test_tweets = []
past = datetime.utcnow() - timedelta(days = 3)
past2 = past - timedelta(days = 7)
# populates the list with only tweet text
i, j = 0, 0
for result in results:
    time_created = datetime.strptime(result['created_at'], '%a %b %d %H:%M:%S %z %Y')
    time_created = time_created.replace(tzinfo = None)
    
    if time_created < past and result['retweeted'] is False and time_created > past2:
        train_tweets.append({})
        train_tweets[i]['created_at'] = time_created
        train_tweets[i]['Tweet'] = result['full_text']
        i += 1
    if time_created > past and result['retweeted'] is False:
        test_tweets.append({})
        test_tweets[j]['created_at'] = time_created
        test_tweets[j]['Tweet'] = result['full_text']
        j += 1

In [15]:
# print a few raw tweets collected
for i in range(0, 5):
    print(train_tweets[i]['Tweet'])

RT @JStanleyFX: video of today's webinar posted and ready to go: US Dollar Price Action Setups in EUR/USD, USD/CAD and USD/CHF https://t.co…
video of today's webinar posted and ready to go: US Dollar Price Action Setups in EUR/USD, USD/CAD and USD/CHF https://t.co/WvVJZun1AJ $USD $EURUSD $USDCAD $USDCHF https://t.co/bHTgnNv4NJ
Haven't posted about Dixie recently, but it's sort of playing out as I expected, though it may be running out of steam, depending on what happens to $EURUSD in the coming week or so. But for now... wheeeee! $USD $DXY #FOREX https://t.co/CLG0uLcNWN
$EURUSD Do you know how to Trade "A Throwover" ? https://t.co/zuY921ZJhQ
RT @commander10: $EURUSD may be forming this inverse H&amp;S on daily chart. Target = 1.1689. #bullish if breaks &amp; holds above. https://t.co/co…


Cleaning Data

In [16]:
regex = '^b[\'"]'
regex_url = "(www|http:|https:|bit)+[^\s]+[\w]"
regex_userTags = '@[a-zA-Z:]+'


i = 0
for dic in train_tweets:
    train_tweets[i]['Tweet'] = re.sub(regex, '', dic['Tweet'])
    train_tweets[i]['Tweet'] = re.sub(regex_url, '', dic['Tweet'])
    train_tweets[i]['Tweet'] = re.sub(regex_userTags, '', dic['Tweet'])
    i += 1

j = 0
for dic in test_tweets:
    test_tweets[j]['Tweet'] = re.sub(regex, '', dic['Tweet'])
    test_tweets[j]['Tweet'] = re.sub(regex_url, '', dic['Tweet'])
    test_tweets[j]['Tweet'] = re.sub(regex_userTags, '', dic['Tweet'])
    j += 1

In [17]:
# print a few cleaned tweets
for i in range(0,5):
    print(train_tweets[i]['Tweet'])

RT  video of today's webinar posted and ready to go: US Dollar Price Action Setups in EUR/USD, USD/CAD and USD/CHF …
video of today's webinar posted and ready to go: US Dollar Price Action Setups in EUR/USD, USD/CAD and USD/CHF  $USD $EURUSD $USDCAD $USDCHF 
Haven't posted about Dixie recently, but it's sort of playing out as I expected, though it may be running out of steam, depending on what happens to $EURUSD in the coming week or so. But for now... wheeeee! $USD $DXY #FOREX 
$EURUSD Do you know how to Trade "A Throwover" ? 
RT 10: $EURUSD may be forming this inverse H&amp;S on daily chart. Target = 1.1689. #bullish if breaks &amp; holds above. …


Creating the labels

In [18]:
import numpy as np
import fxcmpy
from datetime import timedelta

api = fxcmpy.fxcmpy(config_file='fxcm.cfg')
symbol = 'EUR/USD'
labels = []
#data retrieval
for struc in train_tweets:
    start = struc['created_at'] - timedelta(minutes = 1)
    start = start.replace(second = 0)
    stop = struc['created_at'] + timedelta(minutes = 1)
    stop = stop.replace(second = 0)
    candles = api.get_candles(symbol, period = 'm1', start = start, stop = stop)
    try:
        labels.append(np.sign( candles['bidclose'].iloc[1] - candles['bidopen'].iloc[0] ))
    except:
        labels.append(None)
        # do nothing

test_labels = []
for struc in test_tweets:
    start = struc['created_at'] - timedelta(minutes = 1)
    start = start.replace(second = 0)
    stop = struc['created_at'] + timedelta(minutes = 1)
    stop = stop.replace(second = 0)
    candles = api.get_candles(symbol, period = 'm1', start = start, stop = stop)
    try:
        test_labels.append(np.sign( candles['bidclose'].iloc[1] - candles['bidopen'].iloc[0] ))
    except:
        test_labels.append(None)


In [19]:
# print a few labels
print(labels[0:5])

[-1.0, -1.0, -1.0, -1.0, 1.0]


Counting the number of instances for each class

In [37]:
positive = 0
negative = 0
neutral = 0
for it in test_labels:
    if it == 1:
        positive += 1
    if it == 0:
        neutral += 1
    if it == -1:
        negative += 1
print('Number of positives: ' + str(positive))
print('Number of negatives: ' + str(negative))
print('Number of neutrals: ' + str(neutral))

Number of positives: 403
Number of negatives: 386
Number of neutrals: 27


Creating the Document Term matrix

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords

train_tweets_text = []
test_tweets_text = []
labels_shortened = []
# remove tweets that don't have a label
i = 0
for text in train_tweets:
    if labels[i] != None:
        train_tweets_text.append(text['Tweet'])
    i += 1
    
for j in range(0, len(labels)):
    if labels[j] != None:
        labels_shortened.append(labels[j])

for text in test_tweets:
    test_tweets_text.append(text['Tweet'])
    
# remove stop words by using the nltk stop words set
stop_words = set(stopwords.words('english'))
stop_words.add('$')
stop_words.add('&')
stop_words.add('~')
stop_words.add('@')
stop_words.add('#')
stop_words.add('!')
stop_words.add(':')
stop_words.add(',')
stop_words.add('.')
stop_words.add('-')

vectorizer = CountVectorizer(analyzer = 'word', stop_words = stop_words)

train_data_features = vectorizer.fit_transform(train_tweets_text).toarray()

In [23]:
# print train data features
train_data_features

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

Training the Random Forest Classifier

In [24]:
Forest = RandomForestClassifier(n_estimators = 100)
Forest = Forest.fit(train_data_features, labels_shortened)

# vectorizing a test dataset
test_data_features = vectorizer.transform(test_tweets_text).toarray()

In [26]:
# predict the results, using the model
predict = Forest.predict(test_data_features)

Analyzing the Results

In [27]:
i = 0
TP = 0
FP = 0
FN = 0
TN = 0
False_neutral_type1 = 0
False_neutral_type2 = 0
False_neutral_type3 = 0
False_neutral_type4 = 0
True_neutral = 0

for actual in test_labels:
    if actual != None:
        if predict[i] == 1 and actual == 1:
            TP += 1
        if predict[i] == 1 and actual == 0:
            False_neutral_type1 += 1
        if predict[i] == 0 and actual == 1:
            False_neutral_type2 += 1
        if predict[i] == 0 and actual == -1:
            False_neutral_type3 += 1
        if predict[i] == -1 and actual == 0:
            False_neutral_type4 += 1
        if predict[i] == 0 and actual == 0:
            True_neutral += 1
        if predict[i] == -1 and actual == 1:
            FN += 1
        if predict[i] == -1 and actual == -1:
            TN += 1
        if predict[i] == 1 and actual == -1:
            FP += 1
    i += 1

In [35]:
# print Confusion Matrix
print('  1','    0','    -1')
print([TP, False_neutral_type1, FP])
print([False_neutral_type2, True_neutral, False_neutral_type3])
print([FN, False_neutral_type4, TN])

  1     0     -1
[181, 11, 169]
[2, 0, 3]
[220, 16, 214]


In [38]:
print('Accuracy is:')
print((TP + True_neutral + TN)/(TP + True_neutral + 
TN + False_neutral_type1 + FP + False_neutral_type2 + False_neutral_type3 
+ FN + False_neutral_type4))

Accuracy is:
0.4840686274509804


In [42]:
print("Precision is:")
print(181/(181+11+169))
print('Accuracy is:')
print(181/(TP + True_neutral + 
TN + False_neutral_type1 + FP + False_neutral_type2 + False_neutral_type3 
+ FN + False_neutral_type4))

Precision is:
0.5013850415512465
Accuracy is:
0.22181372549019607
